# Coverage Analysis (High Density Scenario)

_Script adapted from full-factorial.ipynb_

In [1]:
import os
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from itertools import product, chain, combinations
from scipy import stats
from IPython.display import display, HTML
%matplotlib inline

def parse_if_number(s):
    try: return float(s)
    except: return True if s=="true" else False if s=="false" else s if s else None

def parse_ndarray(s):
    return np.fromstring(s, sep=' ') if s else None

def get_file_name(name):
    return name.replace(':', '-')

## Config

In [2]:
inputFile = 'coverage.csv'
repetitionsCount = -1 # -1 = auto-detect
factors = ['R', 'T', 'm', 'D']

tIntervalAlpha = 0.9

plotSize = (10, 10)
plotStyle = 'seaborn-whitegrid'
saveFigures = False

# Filter scalars
scalarsFilter = ['Floorplan.userCount', 'Floorplan.coveredUsers:sum']
# Filter vectors
vectorsFilter = []
# Percentiles
percentiles = [0.25, 0.5, 0.75, 0.9, 0.95]

# Performance indexes
perfIndexes = [
    ('coveredUsersPercent', 'percentage of covered users')
]

# Transformations
transformations = [
]

intPercentiles = [int(i*100) for i in percentiles]
vecPerfIndexes = []
#for intPercentile in intPercentiles:
#    vecPerfIndexes.append(('broadcastTime' + str(intPercentile), 'Broadcast time needed to reach the ' + str(intPercentile) + 'th percentile of the coverage'))
for v in vecPerfIndexes:
    perfIndexes.append(v)
    #transformations.append((v[0], lambda x: math.log(x)))

## Load scalars

In [3]:
df = pd.read_csv('exported_data/' + inputFile, converters = {
    'attrvalue': parse_if_number,
    'binedges': parse_ndarray,
    'binvalues': parse_ndarray,
    'vectime': parse_ndarray,
    'vecvalue': parse_ndarray,
})

In [4]:
if repetitionsCount <= 0: # auto-detect
    repetitionsCount = int(df[df.attrname == 'repetition']['attrvalue'].max()) + 1
print('Repetitions:', repetitionsCount)

# Computed
factorsCount = len(factors)

if saveFigures:
    os.makedirs('figures', exist_ok=True)

Repetitions: 10


In [5]:
scalars = df[(df.type == 'scalar') | ((df.type == 'itervar') & (df.attrname != 'TO')) | ((df.type == 'param') & (df.attrname == 'Floorplan.userCount')) | ((df.type == 'runattr') & (df.attrname == 'repetition'))]
scalars = scalars.assign(qname = scalars.attrname.combine_first(scalars.module + '.' + scalars.name))
for index, row in scalars[scalars.type == 'itervar'].iterrows():
    val = scalars.loc[index, 'attrvalue']
    if isinstance(val, str)  and not all(c.isdigit() for c in val):
        scalars.loc[index, 'attrvalue'] = eval(val)
scalars.value = scalars.value.combine_first(scalars.attrvalue.astype('float64'))
scalars_wide = scalars.pivot_table(index=['run'], columns='qname', values='value')
scalars_wide.sort_values([*factors, 'repetition'], inplace=True)
count = 0
for index in scalars_wide.index:
    config = count // repetitionsCount
    scalars_wide.loc[index, 'config'] = config
    count += 1
scalars_wide = scalars_wide[['config', 'repetition', *factors, *scalarsFilter]]

configsCount = int(scalars_wide['config'].max()) + 1
totalSims = configsCount*repetitionsCount
display(HTML("<style>div.output_scroll { height: auto; max-height: 48em; }</style>"))
pd.set_option('display.max_rows', totalSims)
pd.set_option('display.max_columns', 100)

# coverage
scalars_wide['coveredUsersPercent'] = scalars_wide['Floorplan.coveredUsers:sum'] / (scalars_wide['Floorplan.userCount'] - 1)

## Load vectors

In [6]:
vectors = df[df.type == 'vector']
vectors = vectors.assign(qname = vectors.module + '.' + vectors.name)
for index in scalars_wide.index:
    r = index
    cfg = scalars_wide.loc[index, 'config']
    rep = scalars_wide.loc[index, 'repetition']
    vectors.loc[vectors.run == r, 'config'] = cfg
    vectors.loc[vectors.run == r, 'repetition'] = rep
vectors = vectors[vectors.qname.isin(vectorsFilter)]
vectors.sort_values(['config', 'repetition', 'qname'], inplace=True)
vectors = vectors[['config', 'repetition', 'qname', 'vectime', 'vecvalue']]

## Compute scalars from vectors

In [7]:
def get_percentile(percentile, vectime, vecvalue, totalvalue):
    tofind = percentile * totalvalue
    idx = 0
    csum = vecvalue.cumsum()
    for value in csum:
        if value >= tofind:
            return vectime[idx]
        idx += 1
    return math.inf

for index, row in vectors.iterrows():
    for vecPerf, percentile in zip(vecPerfIndexes, percentiles):
        vecPerfIndex = vecPerf[0]
        cfg = row['config']
        rep = row['repetition']
        if vecPerfIndex.startswith('broadcastTime'):
            total = scalars_wide[(scalars_wide['config'] == cfg) & (scalars_wide['repetition'] == rep)]['Floorplan.userCount'].values[0] - 1
        else:
            raise Exception('Need to specify total for ' + vecPerfIndex + '. (coding required)')
        value = get_percentile(percentile, row['vectime'], row['vecvalue'], total)
        scalars_wide.loc[(scalars_wide['config'] == cfg) & (scalars_wide['repetition'] == rep), vecPerfIndex] = value

## Apply transformations

In [8]:
for col, transform in transformations:
    scalars_wide[col] = scalars_wide[col].map(transform, 'ignore')

## Full factorial

In [9]:
for cfg in range(0, configsCount):
    for perfIndex, _ in perfIndexes:
        mean = scalars_wide[scalars_wide['config'] == cfg][perfIndex].mean()
        variance = scalars_wide[scalars_wide['config'] == cfg][perfIndex].var()
        _, positiveInterval = tuple(v*math.sqrt(variance/repetitionsCount) for v in stats.t.interval(tIntervalAlpha, repetitionsCount - 1))
        negerr = positiveInterval
        poserr = positiveInterval
        if perfIndex == 'coveredUsersPercent':
            poserr = min(1 - mean, positiveInterval)
        scalars_wide.loc[scalars_wide['config'] == cfg, perfIndex + 'Mean'] = mean
        scalars_wide.loc[scalars_wide['config'] == cfg, perfIndex + 'Variance'] = variance
        scalars_wide.loc[scalars_wide['config'] == cfg, perfIndex + 'Negerr'] = negerr
        scalars_wide.loc[scalars_wide['config'] == cfg, perfIndex + 'Poserr'] = poserr
scalars_wide = scalars_wide[scalars_wide['repetition'] == 0]

for perfIndex, _ in perfIndexes:
    del scalars_wide[perfIndex]
del scalars_wide['repetition']
del scalars_wide['Floorplan.userCount']
del scalars_wide['Floorplan.coveredUsers:sum']
del scalars_wide['config']

scalars_wide       

qname,R,T,m,D,coveredUsersPercentMean,coveredUsersPercentVariance,coveredUsersPercentNegerr,coveredUsersPercentPoserr
run,,,,,,,,
LowDensityCoverage-0-20210608-13:45:38-13598,10.0,1.0,1.0,1.0,0.000881,9.188028e-07,0.000556,0.000556
LowDensityCoverage-630-20210608-13:56:13-15160,10.0,1.0,1.0,2.0,0.001922,4.871792e-06,0.001279,0.001279
LowDensityCoverage-1260-20210608-14:07:58-16879,10.0,1.0,1.0,3.0,0.001681,4.480054e-06,0.001227,0.001227
LowDensityCoverage-1890-20210608-14:20:26-18669,10.0,1.0,1.0,4.0,0.001201,1.602563e-06,0.000734,0.000734
LowDensityCoverage-2520-20210608-14:32:29-20406,10.0,1.0,1.0,5.0,0.003683,9.999994e-06,0.001833,0.001833
LowDensityCoverage-210-20210608-13:48:09-13970,10.0,1.0,2.0,1.0,0.002962,1.325498e-05,0.002110,0.002110
LowDensityCoverage-840-20210608-13:59:26-15645,10.0,1.0,2.0,2.0,0.002162,2.286323e-06,0.000877,0.000877
LowDensityCoverage-1470-20210608-14:11:08-17331,10.0,1.0,2.0,3.0,0.004644,1.877492e-05,0.002512,0.002512
LowDensityCoverage-2100-20210608-14:23:31-19115,10.0,1.0,2.0,4.0,0.005364,1.724358e-05,0.002407,0.002407


## Coverage results

Here we will print only the rows with a coverage mean > 0.99.

In [10]:
result = scalars_wide[scalars_wide['coveredUsersPercentMean'] >= 0.99]
sortedresult = result.sort_values(factors)
sortedresult

qname,R,T,m,D,coveredUsersPercentMean,coveredUsersPercentVariance,coveredUsersPercentNegerr,coveredUsersPercentPoserr
run,,,,,,,,
LowDensityCoverage-2860-20210608-14:37:35-21260,23.0,1.0,2.0,5.0,0.991914,1.730056e-05,0.002411,0.002411
LowDensityCoverage-2440-20210608-14:29:38-19980,23.0,1.0,3.0,4.0,0.993034,4.423074e-06,0.001219,0.001219
LowDensityCoverage-6010-20210608-15:34:21-30507,23.0,2.0,2.0,5.0,0.991353,3.829057e-05,0.003587,0.003587
LowDensityCoverage-5590-20210608-15:27:07-28873,23.0,2.0,3.0,4.0,0.991433,1.923788e-05,0.002543,0.002543
LowDensityCoverage-6220-20210608-15:38:44-31282,23.0,2.0,3.0,5.0,0.992074,2.485041e-05,0.002890,0.002890
LowDensityCoverage-8740-20210608-16:23:31-6658,23.0,3.0,3.0,4.0,0.990953,1.282763e-05,0.002076,0.002076
LowDensityCoverage-11890-20210608-17:18:31-15327,23.0,4.0,3.0,4.0,0.992634,3.829057e-05,0.003587,0.003587
LowDensityCoverage-12520-20210608-17:29:03-16879,23.0,4.0,3.0,5.0,0.991033,6.507118e-05,0.004676,0.004676
LowDensityCoverage-14830-20210608-18:08:28-23656,23.0,5.0,2.0,4.0,0.990793,2.154557e-05,0.002691,0.002691


In [11]:
for factor in factors:
    print('Minimize', factor)
    current = sortedresult[sortedresult[factor] == sortedresult[factor].min()]
    current = current.sort_values([factor, *[fac for fac in factors if fac != factor]])
    display(current)

Minimize R


qname,R,T,m,D,coveredUsersPercentMean,coveredUsersPercentVariance,coveredUsersPercentNegerr,coveredUsersPercentPoserr
run,,,,,,,,
LowDensityCoverage-2860-20210608-14:37:35-21260,23.0,1.0,2.0,5.0,0.991914,0.000017,0.002411,0.002411
LowDensityCoverage-2440-20210608-14:29:38-19980,23.0,1.0,3.0,4.0,0.993034,0.000004,0.001219,0.001219
LowDensityCoverage-6010-20210608-15:34:21-30507,23.0,2.0,2.0,5.0,0.991353,0.000038,0.003587,0.003587
LowDensityCoverage-5590-20210608-15:27:07-28873,23.0,2.0,3.0,4.0,0.991433,0.000019,0.002543,0.002543
LowDensityCoverage-6220-20210608-15:38:44-31282,23.0,2.0,3.0,5.0,0.992074,0.000025,0.002890,0.002890
LowDensityCoverage-8740-20210608-16:23:31-6658,23.0,3.0,3.0,4.0,0.990953,0.000013,0.002076,0.002076
LowDensityCoverage-11890-20210608-17:18:31-15327,23.0,4.0,3.0,4.0,0.992634,0.000038,0.003587,0.003587
LowDensityCoverage-12520-20210608-17:29:03-16879,23.0,4.0,3.0,5.0,0.991033,0.000065,0.004676,0.004676
LowDensityCoverage-14830-20210608-18:08:28-23656,23.0,5.0,2.0,4.0,0.990793,0.000022,0.002691,0.002691


Minimize T


qname,R,T,m,D,coveredUsersPercentMean,coveredUsersPercentVariance,coveredUsersPercentNegerr,coveredUsersPercentPoserr
run,,,,,,,,
LowDensityCoverage-2860-20210608-14:37:35-21260,23.0,1.0,2.0,5.0,0.991914,0.000017,0.002411,0.002411
LowDensityCoverage-2440-20210608-14:29:38-19980,23.0,1.0,3.0,4.0,0.993034,0.000004,0.001219,0.001219
LowDensityCoverage-1610-20210608-14:13:19-17644,24.0,1.0,2.0,3.0,0.990713,0.000018,0.002477,0.002477
LowDensityCoverage-2240-20210608-14:25:33-19409,24.0,1.0,2.0,4.0,0.993195,0.000016,0.002328,0.002328
LowDensityCoverage-2870-20210608-14:37:52-21323,24.0,1.0,2.0,5.0,0.994956,0.000011,0.001883,0.001883
LowDensityCoverage-1820-20210608-14:17:43-18290,24.0,1.0,3.0,3.0,0.990873,0.000012,0.001972,0.001972
LowDensityCoverage-2450-20210608-14:29:55-20025,24.0,1.0,3.0,4.0,0.995196,0.000010,0.001817,0.001817
LowDensityCoverage-3080-20210608-14:42:33-21996,24.0,1.0,3.0,5.0,0.993595,0.000039,0.003622,0.003622
LowDensityCoverage-1620-20210608-14:13:28-17666,25.0,1.0,2.0,3.0,0.992074,0.000025,0.002873,0.002873


Minimize m


qname,R,T,m,D,coveredUsersPercentMean,coveredUsersPercentVariance,coveredUsersPercentNegerr,coveredUsersPercentPoserr
run,,,,,,,,
LowDensityCoverage-2680-20210608-14:34:36-20746,26.0,1.0,1.0,5.0,0.992634,0.000019,0.002531,0.002531
LowDensityCoverage-2690-20210608-14:34:46-20771,27.0,1.0,1.0,5.0,0.992074,0.000043,0.003786,0.003786
LowDensityCoverage-2070-20210608-14:22:35-18986,28.0,1.0,1.0,4.0,0.993275,0.000008,0.001596,0.001596
LowDensityCoverage-2700-20210608-14:35:04-20815,28.0,1.0,1.0,5.0,0.993995,0.000027,0.002986,0.002986
LowDensityCoverage-2080-20210608-14:23:07-19062,29.0,1.0,1.0,4.0,0.994716,0.000006,0.001421,0.001421
LowDensityCoverage-2710-20210608-14:35:27-20864,29.0,1.0,1.0,5.0,0.994235,0.000018,0.002464,0.002464
LowDensityCoverage-5230-20210608-15:20:32-27936,29.0,2.0,1.0,4.0,0.992314,0.000018,0.002428,0.002428
LowDensityCoverage-5860-20210608-15:32:14-29621,29.0,2.0,1.0,5.0,0.994716,0.000008,0.001669,0.001669
LowDensityCoverage-1460-20210608-14:11:07-17327,30.0,1.0,1.0,3.0,0.994075,0.000012,0.001984,0.001984


Minimize D


qname,R,T,m,D,coveredUsersPercentMean,coveredUsersPercentVariance,coveredUsersPercentNegerr,coveredUsersPercentPoserr
run,,,,,,,,
LowDensityCoverage-10040-20210608-16:47:14-10492,27.0,4.0,3.0,1.0,0.990152,0.000046,0.003945,0.003945
LowDensityCoverage-10050-20210608-16:47:20-10519,28.0,4.0,3.0,1.0,0.991033,0.000010,0.001854,0.001854
LowDensityCoverage-13200-20210608-17:42:06-19260,28.0,5.0,3.0,1.0,0.992634,0.000005,0.001234,0.001234
LowDensityCoverage-6700-20210608-15:48:30-880,29.0,3.0,2.0,1.0,0.991033,0.000010,0.001828,0.001828
LowDensityCoverage-6910-20210608-15:52:56-1829,29.0,3.0,3.0,1.0,0.990953,0.000002,0.000759,0.000759
LowDensityCoverage-9850-20210608-16:43:15-9859,29.0,4.0,2.0,1.0,0.991593,0.000014,0.002169,0.002169
LowDensityCoverage-13210-20210608-17:42:32-19323,29.0,5.0,3.0,1.0,0.991593,0.000009,0.001781,0.001781
LowDensityCoverage-6710-20210608-15:48:48-1039,30.0,3.0,2.0,1.0,0.990953,0.000007,0.001563,0.001563
LowDensityCoverage-6920-20210608-15:53:16-1873,30.0,3.0,3.0,1.0,0.992474,0.000012,0.002043,0.002043


## Observations

As we can see, we need at least R = 7m to get a mean coverage of 99%.

With R = 7m, the lowest config is: R=7m, T=1s, m=2, D=5s. D can be lowered to 3s if we increment T to 4s and m to 3.

With R = 8m we get a lot more possible configurations, but m needs to be at least 2 and D at least 3s. If we use R = 9m, we can have m = 1 and T = 1s but D must be at least 3s.

A "good" and "balanced" config, that does not use the value `1` for any of the parameters, is the one displayed below. From now on, for the high density scenario, we will consider the following as minimum parameters required to get the 99% coverage.

R = 8m

T = 2s

m = 2

D = 2s

In [13]:
display(sortedresult[(sortedresult['R'] == 25) & (sortedresult['T'] == 2) & (sortedresult['m'] == 2) & (sortedresult['D'] == 2)])

qname,R,T,m,D,coveredUsersPercentMean,coveredUsersPercentVariance,coveredUsersPercentNegerr,coveredUsersPercentPoserr
run,,,,,,,,
LowDensityCoverage-4140-20210608-15:00:50-25019,25.0,2.0,2.0,2.0,0.991353,0.000034,0.003381,0.003381
